# Postgres data example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andrewm4894/netdata-gpt-notebooks/blob/main/notebooks/postgres_data_example.ipynb)

In [1]:
# if running in colab uncomment the following line and run it to install the required packages
#!pip install python-dotenv netdata-pandas openai

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from netdata_pandas.data_cloud import get_data_cloud
import openai
import pprint as pp

# load tokens from .env file
load_dotenv()

NETDATA_API_TOKEN = os.getenv('NETDATA_API_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

In [2]:
# inputs

# netdata demo space https://app.netdata.cloud/spaces/netdata-demo/rooms/postgresql/overview
space_id = 'ea93d7b8-0df6-45c0-b13d-1560996c89eb'
room_id = '6a863659-cad0-41af-922d-2c47b0ea0bf2'
contexts = [
    'postgres.connections_utilization',
    'postgres.connections_usage',
    'postgres.connections_state_count',
    'postgres.db_connections_utilization',
    'postgres.transactions_duration',
    'postgres.db_ops_fetched_rows_ratio',
    'postgres.db_ops_read_rows_rate',
    ]

# last 10 minutes of data
after = -60*10
before = 0
freq = '10s' # 10 seconds

In [3]:
df = pd.DataFrame(columns=['time'])

# get data from netdata cloud
for context in contexts:
    df_context = get_data_cloud(space_id, room_id, context, after, before, freq=freq)
    df_context = df_context.add_prefix(f'{context}|')
    df = df.merge(df_context,how='outer',on='time')

df_time_min = df.index.min()
df_time_max = df.index.max()

# save tokens by using t
df['t'] = range(len(df))
df = df.set_index('t')
del df['time']
# try save tokens by using ints
df = (df*100).round(0).ffill().astype('int')
print(df.shape)
df.head()

(59, 12)


,postgres.connections_utilization|used,postgres.connections_usage|available,postgres.connections_usage|used,postgres.connections_state_count|active,postgres.connections_state_count|idle,postgres.connections_state_count|idle_in_transaction,postgres.db_connections_utilization|used,postgres.transactions_duration|100ms,postgres.transactions_duration|500ms,postgres.db_ops_fetched_rows_ratio|fetched,postgres.db_ops_read_rows_rate|fetched,postgres.db_ops_read_rows_rate|returned
t,,,,,,,,,,,,
0,5900,3900,5800,4050,780,970,5900,4441,40,8040,774951,1119401
1,5900,3900,5800,2310,860,2630,5900,4445,0,8190,800716,1133013
2,5900,3900,5800,3900,550,1350,5900,4884,0,8170,796927,1128996
3,5900,3900,5800,1800,1630,2370,5900,4786,0,8180,816372,1161351
4,5900,3900,5800,2620,960,2220,5900,4639,190,8190,819279,1151769


In [4]:
prompt = f"""
You are an experienced SRE and sysadmin.

You are monitoring an postgres database using Netdata Cloud.

Below is the some postgres metrics for this server from {df_time_min} to {df_time_max}, aggregated to a {freq} frequency.

The metrics have all been multiplied by 100 and rounded to the nearest integer to be easier to work with.

Does this server have a problem?

Metrics data:
{df.to_string(max_rows=len(df), max_cols=(len(df.columns)))}
"""
print(prompt)


You are an experienced SRE and sysadmin.

You are monitoring an postgres database using Netdata Cloud.

Below is the some postgres metrics for this server from 0 to 58, aggregated to a 10s frequency.

The metrics have all been multiplied by 100 and rounded to the nearest integer to be easier to work with.

Does this server have a problem?

Metrics data:
    postgres.connections_utilization|used  postgres.connections_usage|available  postgres.connections_usage|used  postgres.connections_state_count|active  postgres.connections_state_count|idle  postgres.connections_state_count|idle_in_transaction  postgres.db_connections_utilization|used  postgres.transactions_duration|100ms  postgres.transactions_duration|500ms  postgres.db_ops_fetched_rows_ratio|fetched  postgres.db_ops_read_rows_rate|fetched  postgres.db_ops_read_rows_rate|returned
t                                                                                                                                                        

In [5]:
# build messages list to pass to openai
messages=[
    {"role": "user", "content": prompt}
]

In [6]:
# call openai api
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
)

In [7]:
reply_content = completion.choices[0].message.content
pp.pprint(reply_content)

('As an AI language model, I cannot definitively say whether or not the server '
 'has a problem as it is dependent on the specific requirements and '
 'performance thresholds of the system. However, as a human expert, you would '
 'need to analyze the metrics and determine if they meet the expected values '
 'and if any abnormalities or trends exist. Some potential red flags could '
 'include high levels of active connections, low availability of connections, '
 'consistently high transaction duration, or low fetched rows ratios. It would '
 'be best to compare these metrics against past values or benchmarked '
 'standards to make any conclusions.')
